# Práctico 3 - Introducción al Aprendizaje Automático

## Modelos de Precios de Mercado
Queremos crear un modelo que prediga si el precio de una acción va a subir o bajar de acuerdo a información del pasado.


### Capital Asset Pricing Model (CAPM)
https://economipedia.com/definiciones/modelo-valoracion-activos-financieros-capm.html
Aplicar a varias acciones.

1. ¿Es fácil encontrar $\alpha$ distinto de cero?

2. ¿Qué interpretación le podemos dar a $\beta$?

3. ¿En qué momento nos gustaría buscar activos con $\beta$ > 1 y $\beta$ < 1? ¿Qué implica tener un $\beta$ < 0?

$$
\alpha + \beta * SPY = KO
$$

In [3]:
from pandas_datareader import data
# Definimos tickers en los cuales estamos interesados
# Algunas empresas del S&P500
tickers = ["MELI"]

# Definimos el rango de fechas deseado
start_date = '2020-01-01'
end_date = '2020-12-31'

# Obtenemos un DataFrame
df = data.DataReader(
    tickers, 
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)["Adj Close"]

In [4]:
df

Symbols,MELI
Date,
2020-01-02,610.190002
2020-01-03,607.799988
2020-01-06,606.549988
2020-01-07,625.000000
2020-01-08,633.440002
...,...
2020-12-24,1690.479980
2020-12-28,1663.719971
2020-12-29,1673.489990


### Modelos Autorregresivos para predecir precios.

https://www.statsmodels.org/stable/examples/notebooks/generated/autoregressions.html

### Árboles de Decisión para predecir suba o baja.
Aplicar GridSearch sobre los parámetros.

Para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando:

- Accuracy
- Precision
- Recall
- F1
- matriz de confusión